## Description
This script creates inputs for NMT model of cdr3b and antigen

1. Parse fasta file (for both snp and indel files)
2. Convert amino acid sequence to overlaping 9 mers
3. save patient files 

In [2]:
library(data.table)
file = "~/project/code/deeplearning/antigen_recognition/data/tcga/mc3_neoantigen/test.snp.pep17.fa"
#     aa = fread(file, sep="\t", header=F,strip.white=FALSE, blank.lines.skip=FALSE)
aa = readLines(file)

In [3]:
# parse.fasta = function(file){

dt = data.table(info=aa[seq(1, length(aa), 8)], 
                mut.aa = aa[seq(2, length(aa), 8)], 
                wt.aa = aa[seq(4, length(aa), 8)],
                mut.aa.pos = aa[seq(3, length(aa), 8)],
                mut.nuc = aa[seq(5, length(aa), 8)], 
                wt.nuc = aa[seq(7, length(aa), 8)], 
                mut.nuc.pos = aa[seq(6, length(aa), 8)]
               )
split.info = function(tt){
    tt =gsub('^>', '',tt)
  strsplit(tt, split="\\|")
}
info.dt = do.call(rbind,split.info(dt$info))
colnames(info.dt) = c("sample","ensmblid", "geneid", "info1", "mutation", "info2")
dt = cbind(dt, info.dt)
dt[,mut.aa:=gsub(' ', '', mut.aa)]
dt[,wt.aa:=gsub(' ', '', wt.aa)]
dt[,mut.aa.pos:=(nchar(mut.aa.pos)-1)/3 + 1]
dt[,mut.nuc.pos:=nchar(mut.nuc.pos)]


### Expression filter

Remove low expressed genes. There are two type of high expression: 
1. If a gene is uniformly highly expressed (> 75% of genes). 
2. If a gene highly expressed in samples *specifically* in samples with mutation (i.e. differtially expressed. ( > 75% of samples) 
 * **The analysis need to be done in cancer specific manner** 
3. Remove the genes with zero expression. 



In [4]:

load("pancancer.exp.mat.RData")

expression.filter = function(dt, overexp.thr = .75){
selected.mutation = normalize.sample.mRNA[dt$geneid, dt$sample] > .75 | 
    normalize.gene.mRNA[dt$geneid, dt$sample] > .75
# dt.back = dt
 dt[which(selected.mutation)]
}


Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'pancancer.exp.mat.RData', probable reason 'No such file or directory'”

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


### Split into 9 mers

In [5]:
split.9mer = function(tt, sample, geneid){
    data.table(sample = sample, geneid = geneid, kmer=sapply(seq(1,nchar(tt)-9+1), 
                                                    function(uu) substr(tt, start=uu,stop=uu+9-1))
                                                    )
#     sapply(seq(mut.pos-9+1,nchar(tt)-9+1), function(uu) list( start=uu,stop=uu+9-1))
}



In [6]:
dt.9mer = do.call(rbind, lapply(seq(nrow(dt)), function(tt) split.9mer(dt$mut.aa[tt],dt$sample[tt], dt$geneid[tt])))

In [7]:
head(dt.9mer,20)

sample,geneid,kmer
TCGA-02-0003-01A-01D-1490-08,TACC2,NIKRKQQDT
TCGA-02-0003-01A-01D-1490-08,TACC2,IKRKQQDTP
TCGA-02-0003-01A-01D-1490-08,TACC2,KRKQQDTPG
TCGA-02-0003-01A-01D-1490-08,TACC2,RKQQDTPGS
TCGA-02-0003-01A-01D-1490-08,TACC2,KQQDTPGSP
TCGA-02-0003-01A-01D-1490-08,TACC2,QQDTPGSPD
TCGA-02-0003-01A-01D-1490-08,TACC2,QDTPGSPDH
TCGA-02-0003-01A-01D-1490-08,TACC2,DTPGSPDHR
TCGA-02-0003-01A-01D-1490-08,TACC2,TPGSPDHRD
TCGA-02-0003-01A-01D-1490-08,SPI1,CLQYPSLSP


## Combine function 

In [9]:
load("normalize.mRNA.RData")
samp.id = substr(colnames(normalize.sample.mRNA), 9, 12)
colnames(normalize.sample.mRNA) = samp.id
colnames(normalize.gene.mRNA) = samp.id

ERROR: Error in is.data.frame(x): object 'normalize.sample.mRNA' not found


### Select samples that have TCR 

In [ ]:
load("tcga_extended_tcr.V3.RData")
tcga_extended_tcr_sel = tcga_extended_tcr[status=="complete" & type=="TRB"]
# tcr.samples = unique(tcga_extended_tcr_sel$sample)
tcga_extended_tcr_sel[,sample.id:=substr(sample, 9 ,12)]
tcr.sample.id = unique(tcga_extended_tcr_sel$sample.id)
common.patients = intersect(colnames(normalize.gene.mRNA), tcr.samples.id)
normalize.gene.mRNA = normalize.gene.mRNA[,common.patients]
normalize.sample.mRNA = normalize.sample.mRNA[,common.patients]

In [8]:
expression.filter = function(dt, overexp.thr = .75){
    dt[,sample:=substr(sample, 9,12)]
    numGenes = nrow(normalize.sample.mRNA)
    
    dt = dt[(geneid %in% rownames(normalize.sample.mRNA)) & (dt$sample %in% colnames(normalize.sample.mRNA)) ]
#     dt = dt[sel]
    midx =  cbind(dt$geneid, dt$sample)
    selected.mutation = normalize.sample.mRNA[midx] > .75 | 
        normalize.gene.mRNA[midx] > .75
    # dt.back = dt
     dt[which(selected.mutation)]
}

split.9mer = function(tt, sample, geneid){
    data.table(sample = sample, geneid = geneid, kmer=sapply(seq(1,nchar(tt)-9+1), 
                                                    function(uu) substr(tt, start=uu,stop=uu+9-1))
                                                    )
}


split2kmers = function(file, overexp.thr = .75, apply.expression.filter =T){

    aa = readLines(file)
    dt = data.table(info=aa[seq(1, length(aa), 8)], 
                    mut.aa = aa[seq(2, length(aa), 8)], 
                    wt.aa = aa[seq(4, length(aa), 8)],
                    mut.aa.pos = aa[seq(3, length(aa), 8)],
                    mut.nuc = aa[seq(5, length(aa), 8)], 
                    wt.nuc = aa[seq(7, length(aa), 8)], 
                    mut.nuc.pos = aa[seq(6, length(aa), 8)]
                   )
    split.info = function(tt){
        tt =gsub('^>', '',tt)
      strsplit(tt, split="\\|")
    }
    info.dt = do.call(rbind,split.info(dt$info))
    colnames(info.dt) = c("sample","ensmblid", "geneid", "info1", "mutation", "info2")
    dt = cbind(dt, info.dt)
    dt[,mut.aa:=gsub(' ', '', mut.aa)]
    dt[,wt.aa:=gsub(' ', '', wt.aa)]
    dt[,mut.aa.pos:=(nchar(mut.aa.pos)-1)/3 + 1]
    dt[,mut.nuc.pos:=nchar(mut.nuc.pos)]
    
    if(apply.expression.filter)
        dt = expression.filter(dt, overexp.thr=overexp.thr)
    
    out = NULL 
    
    if(nrow(dt) > 0)
        out = do.call(rbind, lapply(seq(nrow(dt)),
                                    function(tt) split.9mer(dt$mut.aa[tt],dt$sample[tt], dt$geneid[tt])))
    out
    
}

In [ ]:
snp.files = list.files(path="~/data/neoantigen/run/cromwell-executions/Neoantigen/a10dcf08-3b8a-48f3-970f-87ba2d0558e5/call-get_peptides", 
                          pattern = "test.snp.pep17.fa", recursive = TRUE,  full.names = TRUE)

In [ ]:
library(parallel)
# file = snp.files[1]
# out = split2kmers(file)
out.all = mclapply(snp.files, split2kmers,mc.cores=32)
all.9mers = do.call(rbind, out.all)
save(file="all.9mer.experssion.filter.RData", all.9mers)


### Create kmer sentence files

In [ ]:
my.get.kmers <- function (.data, .head = -1, .k = 5, .clean = T, .verbose = F, .collapse =F, .left.shift = 0, .right.shift = 0) {  
  if (class(.data) == 'list') {
    ngrams <- lapply(.data, get.kmers, .head = .head, .k = .k, .clean = .clean, .collapse = .collapse, .left.shift = .left.shift, .right.shift = .right.shift, .verbose = .verbose)
    return(ngrams)
  }
  
  .n <- .k  
  
  if (.head == -1) {
    .head <- dim(.data)[1]
    if (is.null(.head)) { .head <- length(.data) }
  }
  
  .data <- head(.data, .head)
  
  read.count <- rep.int(1, .head)
  if (class(.data) == 'data.frame') { .data <- .data$CDR3.amino.acid.sequence }
  if (.clean) {
    if (.verbose) cat('Cleaning bad sequences...\t')
    .data <- .data[grep('[*, ~]', .data, invert = T)]
    if (.verbose) cat('Sequences after cleaning:', length(.data),'\n')
  }
  
  if (.verbose) cat('Calculating space...\n')
  .data <- substr(.data, 1 + .left.shift, nchar(.data) - .right.shift)
  non.nchar <- nchar(.data) >= .n
  .data <- .data[non.nchar]
  read.count <- read.count[non.nchar]
  space <- sum(nchar(.data) -.n + 1)
  if (.verbose) cat('Number of k-mers:', space,'\n')
  if (space > 0) {
    if (.verbose) {
      cat('Generating k-mers...\n')
    }
    j <- 1
    ngrams = list()
    for (i in 1:length(.data)) {
       
        temp <- sapply(1:(nchar(.data[i]) - .n + 1), function(j) substr(.data[i], j, j + .n - 1), USE.NAMES=F)
        if(.collapse) temp = paste0(temp, collapse = " ")
        ngrams[[i]] = temp 
      }
    ngrams = unlist(ngrams)
                            
                       
    ngrams
  } else {
    return(NA)
  }
}


In [ ]:
kmer.len = 5
start.pad = paste(rep("^", kmer.len -1 ), collapse = "")
end.pad = paste(rep("$", kmer.len -1 ), collapse = "")
all.9mers[,kmer.padded:=paste0(start.pad, kmer, end.pad)]
# all.9mers$kmer.sentence=unlist(mclapply(all.9mers$kmer.padded, my.get.kmers, .k=5, .collapse=T,  mc.cores =32))
aa = mclapply(seq(13), function(tt) all.9mers[,substr(kmer.padded, tt,tt+4)], mc.cores=32)
command = paste0(paste0("aa[[",1:13,"]]"), collapse=",")
command1 = paste0("paste(",command,",sep=\" \")")
#               command1 = paste0("paste(",command,",sep=\" \")")
bb = eval(parse(text=command1))
all.9mers$kmer.sentence=bb  
all.9mers = all.9mers[order(sample)]
all.9mers$patient = factor(all.9mers$sample)
levels(all.9mers$patient)=seq(length(unique(all.9mers$sample)))
write.table(file="all.9mers.txt",x = all.9mers , row.names = F,col.names =T,  sep="\t", quote=F )


### TCR kmers


In [1]:
split2kmer = function(word, kmer.len){
    aa = lapply(seq(1, nchar(word) - kmer.len + 1), function(tt) substr(word, tt, tt+kmer.len-1))
    paste(unlist(aa), collapse=" ")
}
kmer.len = 5
start.pad = paste(rep("^", kmer.len -1 ), collapse = "")
end.pad = paste(rep("$", kmer.len -1 ), collapse = "")
tcga_extended_tcr_sel[,kmer.padded:=paste0(start.pad, cdr3aa, end.pad)]
tcga_extended_tcr_sel$kmer.sentence = mclapply(tcga_extended_tcr_sel$kmer.padded, split2kmer, kmer.len=5, mc.cores =32) 
tcga_extended_tcr_sel$kmer.sentence = unlist(tcga_extended_tcr_sel$kmer.sentence)

tcga_extended_tcr_final = tcga_extended_tcr_sel[sample.id %in% all.9mers$sample]
tcga_extended_tcr_final =tcga_extended_tcr_final[order(sample.id)]
tcga_extended_tcr_final$patient = factor(tcga_extended_tcr_final$sample.id)
levels(tcga_extended_tcr_final$patient)=seq(length(unique(tcga_extended_tcr_final$sample.id)))
write.table(file="tcga_extended_tcr_final.txt",x = tcga_extended_tcr_final, row.names = F,col.names =T,  sep="\t", quote=F )    
                            

ERROR: Error in mclapply(tcga_extended_tcr_sel$kmer.padded, split2kmer, kmer.len = 5, : could not find function "mclapply"


### 200 patients for small training 

In [ ]:
tcga_extended_tcr_final = fread("tcga_extended_tcr_final.txt")
all.9mers = fread("all.9mers.txt")

sel.patients = sample(unique(all.9mers$patient), size=200)

tcga_extended_tcr_final.sel = tcga_extended_tcr_final[patient %in% sel.patients]
write.table(file="tcga_extended_tcr_final_200.txt",x = tcga_extended_tcr_final.sel, row.names = F,col.names =T,  sep="\t", quote=F )    

all.9mers.sel = all.9mers[patient %in% sel.patients]

write.table(file="all.9mers_200.txt",x = all.9mers.sel, row.names = F,col.names =T,  sep="\t", quote=F )    
                            
